# Probabilidades e análise de texto

Esta é uma prévia para o Projeto 2.

Leia [esta referência do Prof. Sebastian Raschka](https://arxiv.org/pdf/1410.5329.pdf) após concluir a atividade. Também existe [em formato de blog](https://sebastianraschka.com/Articles/2014_naive_bayes_1.html)

A técnica apresentada neste notebook foi usada para resolver [um problema histórico de autoria de documentos](https://priceonomics.com/how-statistics-solved-a-175-year-old-mystery-about/).

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_rows = 13

Abaixo definimos uma função de limpeza simples, que usaremos nos trechos de texto analizados

In [ ]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed
    

## Análise "os Lusíadas" vs "Dom Casmurro"

Vamos analisar as obras Dom Casmurro, de [Machado de Assis](http://machado.mec.gov.br/) e Os Lusíadas, de [Luis Vaz de Camões](https://pt.wikipedia.org/wiki/Lu%C3%ADs_de_Cam%C3%B5es). Ambas as obras são de domínio público e foram obtidas no site do [Projeto Gutenberg](https://www.gutenberg.org/wiki/PT_Principal)

In [ ]:
lusiadas_raw = open("textos/lusiadas_texto.txt", "r").read()

In [ ]:
casmurro_raw = open("textos/domcasmurro_texto.txt", "r").read()

### Limpezas das bases

Vamos  converter todo o texto para minúsculas, para facilitar a análise de frequência das palavras. Também vamos aplicar uma limpeza rudimentar de pontuação com a função `cleanup()` definida anteriormente.

In [ ]:
lusiadas = cleanup(lusiadas_raw.lower())

In [ ]:
casmurro = cleanup(casmurro_raw.lower())

Vamos inspecionar  os arquivos

In [ ]:
print(lusiadas[0:100])

In [ ]:
print(casmurro[0:100])

### Tabelas de frequências

Vamos construir a tabela de frequência absoluta para as obras

Primeiramente vamos converter as obras objetos do tipo `pd.Series`

In [ ]:
serie_casmurro = pd.Series(casmurro.split())

In [ ]:
serie_casmurro

Palavras em um texto são variáveis **qualitativas nominais**, portanto usaremos `value_counts()` para obter a tabela de frequências relativas e absolutas

In [ ]:
tabela_casmurro = serie_casmurro.value_counts()

#### Frequências absolutas

In [ ]:
tabela_casmurro

Frequências relativas

In [ ]:
tabela_casmurro_relativa = serie_casmurro.value_counts(True)

#### Frequências relativas

In [ ]:
tabela_casmurro_relativa

É de se estranhar a linha abaixo?

In [ ]:
tabela_casmurro_relativa.sum()

#### Fazendo um loop na Series

Note que se pode facilmente fazer um loop sobre uma `Series`, que se comporta de forma similar a um dicionário:

In [ ]:
imprime = 4
for palavra in tabela_casmurro_relativa.index:
    if imprime:
        print(tabela_casmurro_relativa[palavra])
        imprime-=1
    


Agora faremos exatamente os mesmos passos de converter em `Series` e obter as tabelas de frequência para a obra "Os Lusíadas"

### Tabelas para Os Lusíadas

In [ ]:
serie_lusiadas = pd.Series(lusiadas.split())

In [ ]:
tabela_lusiadas = serie_lusiadas.value_counts()

In [ ]:
tabela_lusiadas_relativa = serie_lusiadas.value_counts(True)

In [ ]:
tabela_lusiadas

In [ ]:
tabela_lusiadas_relativa

## Probabilidades

Vamos assumir para fins destas análises que a frequência relativa observada nestes textos é igual à probabilidade. Ou seja, que se quisermos encontrar

$P(portugal|Lusíadas)$ basta checar o valor de `tabela_lusiadas_relativa["portugal"]`, que é o que faremos abaixo:

In [ ]:
tabela_lusiadas_relativa["portugal"]

Da mesma fora, se quisermos $P(capitú|casmurro)$ basta checar: 

In [ ]:
tabela_casmurro_relativa["capitú"]

Notamos que na versão do *Gutenberg* a palavra *Capitú* aparece grafada com acento.

## Probabilidades na língua portuguesa

Vamos assumir que o todo da língua portuguesa fosse formado pela fusão das obras *Dom Casmurro* e *Os Lusíadas*

In [ ]:
portugues = lusiadas + casmurro

Podemos refazer a análise de probabilidades considerando este novo *corpus* de texto

In [ ]:
serie_portugues = pd.Series(portugues.split())

In [ ]:
tabela_portugues_relativa = serie_portugues.value_counts(True)

In [ ]:
tabela_portugues_relativa

Desta forma, se quisermos saber a probabilidade da palavra *pintura* em toda a língua portuguesa, a notação seria simplesmente $P(pintura)$ porque estamos a assumir que trabalhamos com a totalidade da língua, ou seja **o conjunto universo**.

E a probabilidade $P(pintura)$ é:

In [ ]:
tabela_portugues_relativa["pintura"]

**Interseção entre os conjuntos**

Podemos obter a interseção entre os conjuntos usando a classe `set` do Python

In [ ]:
set_casmurro = set(tabela_casmurro_relativa.index)
set_lusiadas = set(tabela_lusiadas_relativa.index)
inter = set_casmurro.intersection(set_lusiadas)

Descomente a linha abaixo se quiser ver um `print` da interseção

In [ ]:
# inter

## Classificação: Lusíadas ou Dom Casmurro?

Agora vamos ao problema que queremos resolver.

Você precisa dizer se a frase *"Contou que João como santo se verá vestido, de maneira  que virão a barba do marido"* é mais provável de ter vindo de *Os Lusíadas* ou de *Dom Casmurro*

Ou seja, precisa decidir se:

$P(Casmurro|frase) > P(Lusíadas|frase)$

Vamos indicar $Casmurro$ como $C$ e $Lusíadas$ como $L$ para brevidade.

O teorema de Bayes vai ser particularmente útil neste caso. Lembre-se que:

$P(C|frase) = \frac{P(frase|C)P(C)}{P(frase)}$

e que:

$P(L|frase) = \frac{P(frase|L)P(L)}{P(frase)}$

### Frase a classificar

In [ ]:
frase = "Contou que João como santo se verá vestido, de maneira  que virão a barba do marido"

Vamos converter primeiro em minúsculas e fazer a limpeza

In [ ]:
frase = cleanup(frase.lower())

Agora a frase está assim

In [ ]:
frase

Em forma de lista, para facilitar o processamento:

In [ ]:
frase.split()

### A ingenuidade - Naïve Bayes

Agora vamos à parte ingênua do Naïve Bayes, que consiste em assumir que as palavras são independentes entre si e que sua ordem na frase não importa. 


Ou seja:

$P(frase|C) = 
P(contou|C).P(que|C).P(joão|C).P(como|C).P(santo|C).P(se|C).P(verá|C).P(vestido|C).P(de|C).P(maneira|C).P(que|C).P(virão|C).P(a|C).P(barba|C).P(do|C).P(marido|C)$


Vamos denotar $Casmurro$ simplesmente como $C$, para encurtar a fórmula




A fórmula completa fica então:

$P(C|frase) = \frac{P(contou|C).P(que|C).P(joão|C).P(como|C).P(santo|C).P(se|C).P(verá|C).P(vestido|C).P(de|C).P(maneira|C).P(que|C).P(virão|C).P(a|C).P(barba|C).P(do|C).P(marido|C).P(C)}{P(frase)}$



Da mesma forma, denotando *Os Lusíadas* como $L$ a fórmula completa fica:

$P(L|frase) = \frac{P(contou|L).P(que|L).P(joão|L).P(como|L).P(santo|L).P(se|L).P(verá|L).P(vestido|L).P(de|L).P(maneira|L).P(que|L).P(virão|L).P(a|L).P(barba|L).P(do|L).P(marido|L).P(L)}{P(frase)}$


Note que precisamos somente classificar se $P(C|frase) > P(L|frase)$, de modo que podemos cancelar o denominador $P(frase)$ que aparece em ambos

# O que você deve fazer

**1** Como podemos calcular os valores dos priors $P(C)$ e $P(L)$ ? Ou colocando a pergunta em termos Bayesianos: o que é um *prior* razoável para se usar?

**Dica**:

Você pode usar proporção de número de palavras do conjunto *Lusíadas* e *Dom Casmurro* em relação ao total. Ou ainda proporção destas palavras ponderadas pela frequência absoluta



**2.** Calcule os termos: 

$$P(contou|L).P(que|L).P(joão|L).P(como|L).P(santo|L).P(se|L).P(verá|L).P(vestido|L).P(de|L).P(maneira|L).P(que|L).P(virão|L).P(a|L).P(barba|L).P(do|L).P(marido|L)$$

e 

$$
P(frase|C) = P(contou|C).P(que|C).P(joão|C).P(como|C).P(santo|C).P(se|C).P(verá|C).P(vestido|C).P(de|C).P(maneira|C).P(que|C).P(virão|C).P(a|C).P(barba|C).P(do|C).P(marido|C)$$

**R.:** 

**3.** Calcule $P(L|frase)$ e $P(C|frase)$ como indicado acima

**R.:** 

**4.** Escreva seu parecer: você acha que o texto indicado pertence a Os Lusíadas ou a Dom Casmurro?

**R.:** responda aqui

# O que você deve pesquisar para fazer o Projeto

No [texto recomendado](https://arxiv.org/pdf/1410.5329.pdf) estude o que fazer quando duas situações acontecerem:
- Quando aparecem palavras inéditas para classificar
- Quando a multiplicação das probabilidades é um valor tão pequeno que ocorre *underflow*